<a href="https://colab.research.google.com/github/10100111/Display-of-HW1/blob/main/47_Lite_%D0%9D%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D1%8B%D0%B5_%D1%81%D0%B5%D1%82%D0%B8_13_%D0%93%D0%90_%D0%A7%D0%B0%D1%81%D1%82%D1%8C_2_%D0%9F%D0%BE%D0%B4%D0%B1%D0%BE%D1%80_%D0%B0%D1%80%D1%85%D0%B8%D1%82%D0%B5%D0%BA%D1%82%D1%83%D1%80%D1%8B%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Инфо по выделенным ресурсам


In [ ]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install -q gputil
!pip install -q psutil
!pip install -q humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
process = psutil.Process(os.getpid())
print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
print("GPU Model: ", gpu.name)

Gen RAM Free: 24.5 GB  | Proc size: 3.0 GB
GPU RAM Free: 15561MB | Used: 719MB | Util   4% | Total 16280MB
GPU Model:  Tesla P100-PCIE-16GB


# Генетические алгоритмы для обучения нейронных сетей

*Разбор данного раздела:* https://youtu.be/njAH6ASF0H0?t=389

Чтобы запускать и редактировать код, сохраните копию этого ноутбука себе (File->Save a copy in Drive...). Свою копию вы сможете изменять и запускать.

In [ ]:
#Загружаем библиотеки

import pandas as pd # Пандас
import matplotlib.pyplot as plt # Отрисовка графиков
from tensorflow.keras import utils # Для to_categorical
import numpy as np # Numpy
from tensorflow.keras.optimizers import Adam # Оптимизатор
from tensorflow.keras.models import Sequential, Model # Два варианты моделей
from tensorflow.keras.layers import concatenate,Reshape, Input,Conv2DTranspose, Lambda, Dense, Dropout, BatchNormalization, Flatten, Conv1D, Conv2D, LSTM, MaxPooling1D, Activation, GlobalMaxPooling1D, UpSampling1D#Стандартные слои
from tensorflow.keras.losses import MAE
from google.colab import files # Загрузка файлов
from sklearn.preprocessing import StandardScaler, MinMaxScaler # Нормировщики
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator # Для генерации выборки временных рядов
import tensorflow.keras.backend as K
import random as random # Импортируем модель randim
import numpy as np # Импортируем библиотеку numpy
import matplotlib.pyplot as plt # Импортируем модуль pyplot библиотеки matplotlib для построения графиков
from PIL import Image # Импортируем одель Image для обработки изображений
import time
from tensorflow.keras.utils import plot_model
# Статический вывод графики (графики отображаются в той ячейке, в которой используется plt.show())
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Подбор архитектуры свёрточной сети

#Прогнозирование трафика

*Разбор данного раздела:* https://youtu.be/njAH6ASF0H0?t=542

In [ ]:
# Считываем данные с помощью pandas
base_data = pd.read_csv("/content/drive/MyDrive/Базы/traff.csv", header=None)

In [ ]:
# Выводим пять первых строк
base_data

,0,1
0,1/1/2017,"22,226.00"
1,1/2/2017,"29,593.00"
2,1/3/2017,"31,726.00"
3,1/4/2017,"32,684.00"
4,1/5/2017,"31,820.00"
...,...,...
1090,12/27/2019,"29,801.00"
1091,12/28/2019,"34,913.00"
1092,12/29/2019,"37,665.00"
1093,12/30/2019,"32,643.00"


In [ ]:
data = base_data.iloc[:,1]

In [ ]:
# Выводим параметры одного фала
d = data
print(len(d))         # Сколько есть записей
print(len(d.iloc[0])) # Длинна одной строки данных
print(d.iloc[0])      # Пример первой строки данных

1095
9
22,226.00


In [ ]:
data = np.array(data) #Превращаем в numpy массив
for i in range(len(data)):
  data[i] = float(data[i].replace(',',''))

**Функция загрузки данных**

In [ ]:
#Формируем параметры загрузки данных
xLen = 60     # Анализируем по 300 прошедшим точкам 
valLen = 300  # Используем 300 записей для проверки

trainLen = data.shape[0]-valLen # Размер тренировочной выборки

# Делим данные на тренировочную и тестовую выборки 
Train,Test = np.reshape(data[:trainLen],(-1,1)), np.reshape(data[trainLen+xLen+2:],(-1,1))

# Масштабируем данные (отдельно для X и Y), чтобы их легче было скормить сетке
Scaler = MinMaxScaler()
Scaler.fit(Train)
Train = Scaler.transform(Train)
Test = Scaler.transform(Test)


# Создаем генератор для обучения
trainDataGen = TimeseriesGenerator(Train, Train,             # в качестве параметров наши выборки
                               length=xLen, sampling_rate=1, # для каждой точки
                               batch_size=20)                # размер batch, который будем скармливать модели

# Создаем аналогичный генератор для валидации при обучении
testDataGen = TimeseriesGenerator(Test, Test,
                               length=xLen, sampling_rate=1,
                               batch_size=20)

**Функции для отображения результатов**

In [ ]:
# Функция рассчитываем результаты прогнозирования сети
# В аргументы принимает сеть (currModel) и проверочную выборку
# Выдаёт результаты предсказания predVal
# И правильные ответы в исходной размерности yValUnscaled (какими они были до нормирования)
def getPred(currModel, xVal, yVal, yScaler):
  # Предсказываем ответ сети по проверочной выборке
  # И возвращаем исходны масштаб данных, до нормализации
  predVal = yScaler.inverse_transform(currModel.predict(xVal))
  yValUnscaled = yScaler.inverse_transform(yVal)
  
  return (predVal, yValUnscaled)

# Функция визуализирует графики, что предсказала сеть и какие были правильные ответы
# start - точка с которой начинаем отрисовку графика
# step - длина графика, которую отрисовываем
# channel - какой канал отрисовываем
def showPredict(start, step, channel, predVal, yValUnscaled):
  plt.plot(predVal[start:start+step, channel], 
           label='Прогноз')
  plt.plot(yValUnscaled[start:start+step, channel], 
           label='Базовый ряд')
  plt.xlabel('Время')
  plt.ylabel('Значение Close')
  plt.legend()
  plt.show()
  
# Функция расёта корреляции дух одномерных векторов
def correlate(a, b):
  # Рассчитываем основные показатели
  ma = a.mean() # Среднее значение первого вектора
  mb = b.mean() # Среднее значение второго вектора
  mab = (a*b).mean() # Среднее значение произведения векторов
  sa = a.std() # Среднеквадратичное отклонение первого вектора
  sb = b.std() # Среднеквадратичное отклонение второго вектора
  
  #Рассчитываем корреляцию
  val = 0
  if ((sa>0) & (sb>0)):
    val = (mab-ma*mb)/(sa*sb)
  return val

# Функция рисуем корреляцию прогнозированного сигнала с правильным
# Смещая на различное количество шагов назад
# Для проверки появления эффекта автокорреляции
# channels - по каким каналам отображать корреляцию
# corrSteps - на какое количество шагов смещать сигнал назад для рассчёта корреляции
# showGraf - показываем график или нет
# returnData - возвращаем массивы автокорреляции или нет
def autoCorr(channels, corrSteps, predVal, yValUnscaled, showGraf = True, returnData = False):
  # Проходим по всем каналам
  for ch in channels:
    corr = [] # Создаём пустой лист, в нём будут корреляции при смезении на i рагов обратно
    yLen = yValUnscaled.shape[0] # Запоминаем размер проверочной выборки

      # Постепенно увеличикаем шаг, насколько смещаем сигнал для проверки автокорреляции
    for i in range(corrSteps):
      # Получаем сигнал, смещённый на i шагов назад
      # predVal[i:, ch]
      # Сравниваем его с верными ответами, без смещения назад
      # yValUnscaled[:yLen-i,ch]
      # Рассчитываем их корреляцию и добавляем в лист
      corr.append(correlate(yValUnscaled[:yLen-i,ch], predVal[i:, ch]))

    own_corr = [] # Создаём пустой лист, в нём будут корреляции при смезении на i рагов обратно

      # Постепенно увеличикаем шаг, насколько смещаем сигнал для проверки автокорреляции
    for i in range(corrSteps):
      # Получаем сигнал, смещённый на i шагов назад
      # predVal[i:, ch]
      # Сравниваем его с верными ответами, без смещения назад
      # yValUnscaled[:yLen-i,ch]
      # Рассчитываем их корреляцию и добавляем в лист
      own_corr.append(correlate(yValUnscaled[:yLen-i,ch], yValUnscaled[i:, ch]))

    # Отображаем график коррелций для данного шага
    if showGraf: #Если нужно показать график
      plt.plot(corr, label='предсказание на ' + str(ch+1) + ' шаг')
      plt.plot(own_corr, label='Эталон')

  if showGraf: #Если нужно показать график
    plt.xlabel('Время')
    plt.ylabel('Значение')
    plt.legend()
    plt.show()

  if returnData: #Если нужно вернуть массивы автокорреляции
    return corr, own_corr

**Генерируем проверочную выборку**

In [ ]:
# Создадим генератор проверочной выборки, из которой потом вытащим xVal, yVal для проверки
DataGen = TimeseriesGenerator(Test, Test,
                               length=xLen, sampling_rate=1,
                               batch_size=len(Test)) # Размер batch будет равен длине нашей выборки

xVal = []
yVal = []
for i in DataGen:
  xVal.append(i[0])
  yVal.append(i[1])

xVal = np.array(xVal)
yVal = np.array(yVal)

# Подбор линейной архитектуры

*Разбор данного раздела:* https://youtu.be/njAH6ASF0H0?t=706

In [ ]:
#Создаём сеть (net - список параметров) !Изменено
def createConvNet(net):

  model = Sequential()             # Создаем моель Sequential

  blocksNumber = net[0]  # Делаем ли нормализацию в начале


  activation_list = ['linear','relu','tanh','softmax','sigmoid']
  index_net = 1

  for block in range(blocksNumber): 
    extent = net[index_net]
    ConvSize = 2 ** extent      # Размер первого вёрточного слоя
    print(index_net, net[index_net], ConvSize, 'ConvSize')

    index_net += 1  
    ConvKernel = net[index_net]         # Ядро первого свёрточного слоя
    print(index_net, net[index_net], ConvKernel, 'ConvKernel')
    
    index_net += 1
    activation = net[index_net]             # Функция активации входного слоя
    print(index_net, net[index_net], activation, 'activation')
    
    index_net += 1
    makeMaxPooling = net[index_net]         # Делаем ли maxpooling для нулевого слоя
    print(index_net, net[index_net], makeMaxPooling, 'makeMaxPooling')

    index_net += 1
    maxPoolingSize = net[index_net]         # Размер MaxPooling
    print(index_net, net[index_net], maxPoolingSize, 'maxPoolingSize')
    
    index_net += 1
    
    if block == 0:
      model.add(Conv1D(ConvSize, ConvKernel, input_shape=(xLen, 1), activation=activation_list[activation], padding='same'))
    else:
      model.add(Conv1D(ConvSize, ConvKernel, activation=activation_list[activation], padding='same'))
    if (makeMaxPooling):           # Если делаем MaxPooling
      # Добавляем слой MaxPooling1D с размером (maxPoolingSize)
      model.add(MaxPooling1D(pool_size=maxPoolingSize))



  activation3 = net[index_net]            # Функция активации для   
  print(index_net, net[index_net], activation3, 'activation3')

  index_net += 1
  activation4 = net[index_net]            # Функция активации для последнего слоя
  print(index_net, net[index_net], activation4, 'activation4', type(index_net))

  denseSize = 2 ** net[index_net]          # Размер полносвязного слоя

  model.add(Flatten())              # Добавляем слой Flatten

  model.add(Dense(denseSize, activation=activation_list[activation3])) # Добавляем слой Dense с denseSize нейронами
  model.add(Dense(1, activation=activation_list[activation4]))         # Добавляем Dense-слой с softmax-активацией и 10 нейронами

  return model                      # Возвращаем модель

  """
Функция вычисления результата работы сети
  """

def evaluateNet(net, ep, verb):
  val = 0
  time.time()
  model = createConvNet(net) # создаем модель 

  # Компилируем модель
  model.compile(optimizer=Adam(learning_rate=1e-4),
                loss="mse")
  
  history = model.fit_generator(trainDataGen, 
                                epochs=ep, #???! тут должно быть epochs=ep
                                verbose=verb,
                                validation_data=testDataGen)
  
  val = history.history["val_loss"][-1] # возвращаем точность на проверочной выборке с последней эпохи

  return val, model # возвращаем точность

'''
  Функция создания списка случайных параметров  
'''
def createRandomNet(): #!Изменено
   
  net = []
  net.append(random.randint(1,5)) # Сколько делаем слоев

  blocks = net[0]
  for block in range(blocks):
    net.append(random.randint(3,6)) # свёрточный слой от 8 до 64 нейронов
    net.append(random.randint(3,7)) # Ядро свёрточного слоя от 3 до 7
    net.append(random.randint(0,4)) # Функция активации слоя
    net.append(random.randint(0,1)) #Делаем ли MaxPooling
    net.append(random.randint(2,3)) #Размер MaxPoolin

  net.append(random.randint(0,4)) # Функция активации предпоследнего dense слоя
  net.append(random.randint(0,2)) # Функция активации последнего слоя

  net.append(random.randint(3,6)) # Размер полносвязного слоя от 8 до 64

  if len(net) < 29:
    delta = l - len(net) # !Изменено
    for j in range(delta):
      net.append(0)
    
  return net     

## Проверка кода - ошибка 

In [ ]:
# Код из функции createRandomNet() отрабатывает
   
net = []
net.append(random.randint(1,5)) # Сколько делаем слоев

blocks = net[0]
for block in range(blocks):
  net.append(random.randint(3,6)) # свёрточный слой от 8 до 64 нейронов
  net.append(random.randint(3,7)) # Ядро свёрточного слоя от 3 до 7
  net.append(random.randint(0,4)) # Функция активации слоя
  net.append(random.randint(0,1)) #Делаем ли MaxPooling
  net.append(random.randint(2,3)) #Размер MaxPoolin

net.append(random.randint(0,4)) # Функция активации предпоследнего dense слоя
net.append(random.randint(0,2)) # Функция активации последнего слоя

net.append(random.randint(3,6)) # Размер полносвязного слоя от 8 до 64

if len(net) < 29:
  delta = l - len(net) # !Изменено
  for j in range(delta):
    net.append(0)
    
 

In [ ]:
print(net, '\n', len(net), '\n', net[0], '\n',type(net[0]))

[5, 5, 5, 2, 0, 3, 5, 5, 2, 1, 3, 4, 5, 1, 0, 3, 5, 4, 1, 0, 3, 3, 4, 4, 0, 3, 2, 1, 6] 
 29 
 5 
 <class 'int'>


In [ ]:
# проверяю код из функции createConvNet(net), распечатываю сформированные слои 

model = Sequential()             # Создаем моель Sequential

blocksNumber = net[0]  # Делаем ли нормализацию в начале


activation_list = ['linear','relu','tanh','softmax','sigmoid']
index_net = 1

for block in range(blocksNumber): 
  extent = net[index_net]
  ConvSize = 2 ** extent      # Размер первого вёрточного слоя
  print(index_net, net[index_net], ConvSize, 'ConvSize')

  index_net += 1  
  ConvKernel = net[index_net]         # Ядро первого свёрточного слоя
  print(index_net, net[index_net], ConvKernel, 'ConvKernel')
  
  index_net += 1
  activation = net[index_net]             # Функция активации входного слоя
  print(index_net, net[index_net], activation, 'activation')
  
  index_net += 1
  makeMaxPooling = net[index_net]         # Делаем ли maxpooling для нулевого слоя
  print(index_net, net[index_net], makeMaxPooling, 'makeMaxPooling')

  index_net += 1
  maxPoolingSize = net[index_net]         # Размер MaxPooling
  print(index_net, net[index_net], maxPoolingSize, 'maxPoolingSize')
  
  index_net += 1
  
  if block == 0:
    model.add(Conv1D(ConvSize, ConvKernel, input_shape=(xLen, 1), activation=activation_list[activation], padding='same'))
  else:
    model.add(Conv1D(ConvSize, ConvKernel, activation=activation_list[activation], padding='same'))
  if (makeMaxPooling):           # Если делаем MaxPooling
    # Добавляем слой MaxPooling1D с размером (maxPoolingSize)
    model.add(MaxPooling1D(pool_size=maxPoolingSize))



activation3 = net[index_net]            # Функция активации для   
print(index_net, net[index_net], activation3, 'activation3')

index_net += 1
activation4 = net[index_net]            # Функция активации для последнего слоя
print(index_net, net[index_net], activation4, 'activation4', type(index_net))

denseSize = 2 ** net[index_net]          # Размер полносвязного слоя

model.add(Flatten())              # Добавляем слой Flatten

model.add(Dense(denseSize, activation=activation_list[activation3])) # Добавляем слой Dense с denseSize нейронами
model.add(Dense(1, activation=activation_list[activation4]))         # Добавляем Dense-слой с softmax-активацией и 10 нейронами




1 5 32 ConvSize
2 5 5 ConvKernel
3 2 2 activation
4 0 0 makeMaxPooling
5 3 3 maxPoolingSize
6 5 32 ConvSize
7 5 5 ConvKernel
8 2 2 activation
9 1 1 makeMaxPooling
10 3 3 maxPoolingSize
11 4 16 ConvSize
12 5 5 ConvKernel
13 1 1 activation
14 0 0 makeMaxPooling
15 3 3 maxPoolingSize
16 5 32 ConvSize
17 4 4 ConvKernel
18 1 1 activation
19 0 0 makeMaxPooling
20 3 3 maxPoolingSize
21 3 8 ConvSize
22 4 4 ConvKernel
23 4 4 activation
24 0 0 makeMaxPooling
25 3 3 maxPoolingSize
26 2 2 activation3
27 1 1 activation4 <class 'int'>


In [ ]:
model.compile(optimizer=Adam(learning_rate=1e-4),
                loss='mse')
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 60, 32)            192       
                                                                 
 conv1d_3 (Conv1D)           (None, 60, 32)            5152      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 20, 32)           0         
 1D)                                                             
                                                                 
 conv1d_4 (Conv1D)           (None, 20, 16)            2576      
                                                                 
 conv1d_5 (Conv1D)           (None, 20, 32)            2080      
                                                                 
 conv1d_6 (Conv1D)           (None, 20, 8)             1032      
                                                      

Примечание: Код из функции по сборке нейронки работает , можно задать любое количество слоев. Но что-то пошло пошло не так с общим пайплайном, прошу при снижении оценки внести исправление в мой код так, чтобы пайплайн заработал, буду благодарна. Мне очень нужно разобраться, в разборе вариант видела, списывать не буду, хочу понять как могло бы сработать в моем случае.

In [ ]:
n = 20 # общее число ботов
nsurv = 10 # к-во выживших
nnew = n - nsurv # к-во новых ботов
l = 29 # размер бота !Изменено закомментировано
 
epochs = 10 

mut = 0.09

popul = [] # массив популяции
val = [] # одномерный массив значений этих ботов

# Создаем случайных ботов
for i in range(n): # пробегаемся по всем ботам , их 20
  popul.append(createRandomNet()) # генерим случайные значения в векторе каждого бота и добавляем бота в массив
  delta = l - len(popul) # !Изменено
  for j in range(delta):
    popul.append(0)
    
for it in range(epochs): # пробегаем по всем эпохам
  val = []               # обнуляем значение бота
  curr_time = time.time()# засекаем время
  for i in range(n):     # пробегаем в цикле по всем ботам
    bot = popul[i]       # берем очередного бота
    f, model_sum = evaluateNet(bot, 3, 0) # вычисляем точность текущего бота на 3х эпохах, скрыть вывод данных verbose 
    val.append(f)        # добавляем полученное значение к списку val

  sval = sorted(val, reverse=0) # сортируем val
  # Выводим 5 лучших ботов
  print(it, time.time() - curr_time, " ", sval[0:5],popul[:5])

  newpopul = []         # создаем пустой спиок под новую популяцию
  for i in range(nsurv):# пробегаем по всем выжившим ботам
    index = val.index(sval[i]) # получаем индекс лучшего бота из списка лучших в списке val
    newpopul.append(popul[index]) # добавляем в новую популяцию бота из popul c индексом index

  for i in range(nnew): # проходимся в цикле nnew раз - сколько нам надо сделать ботов
    indexp1 = random.randint(0, nsurv-1) # случайный индекс 1го родителя в диапазоне от 0 до nsurv-1
    indexp2 = random.randint(0, nsurv-1) # случайный индекс 2го родителя в диапазоне от 0 до nsurv-1
    botp1 = newpopul[indexp1] # 1й родитель бота
    botp2 = newpopul[indexp2] # 2й родитель бота
    newbot = []               # создаем пустой список под значения нового бота
    net4Mut = createRandomNet() # создаем случайную сеть для мутаций !Изменено
    for j in range(l):        # пробегаемся по всей длине бота
      x = 0
      pindex = random.random() # получаем случайное число от 0 до 1
      # если pindex меньше 0.5, то берем значения от первого бота, иначе от второго
      if pindex < 0.5:
        x = botp1[j]
      else:
        x = botp2[j]

      # c вероятностью mut устанавливаем значение бота из net4Mut
      if (random.random() < mut):
        x = net4Mut[j]

      newbot.append(x)       # добавляем очередное значение в бота
    newpopul.append(newbot)  # добавляем бота в новую популяцию
    delta = l - len(newpopul) # !Изменено
    for j in range(delta):
      popul.append(0)
  popul = newpopul           # записываем в popul новую посчитанную популяцию
  



1 3 8 ConvSize
2 4 4 ConvKernel
3 4 4 activation
4 0 0 makeMaxPooling
5 2 2 maxPoolingSize
6 4 4 activation3
7 1 1 activation4 <class 'int'>


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:77: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


TypeError: ignored

In [ ]:
print(popul) #Геномы лучших ботов по эпохам

In [ ]:
model = createConvNet(popul[0])
# model.summary()

In [ ]:
#Комипилируем и обучаем
model.compile(optimizer=Adam(lr=1e-4),
                loss='mse')

history = model.fit_generator(trainDataGen,
                    epochs=50, 
                    validation_data=testDataGen)

In [ ]:
model.summary()

In [ ]:
plt.plot(history.history['loss'], 
         label='Средняя абсолютная ошибка на обучающем наборе')
plt.plot(history.history['val_loss'], 
         label='Средняя абсолютная ошибка на проверочном наборе')
plt.ylabel('Средняя ошибка')
plt.legend()
plt.show()

In [ ]:
# Прогнозируем данные текущей сетью
currModel = model #Выбираем текущую модель
(predVal, yValUnscaled) = getPred(currModel, xVal[0], yVal[0], Scaler) #Прогнозируем данные

# Отображаем графики
showPredict(0, 400, 0, predVal, yValUnscaled)

In [ ]:
# Отображаем корреляцию
# Используем выходной канал - только open
# Проверяем корреляцию на глубину 60 шагов
autoCorr([0], 20, predVal, yValUnscaled)